# DSA210 Project – Apple vs Samsung Stock Analysis (2019–2024)

This notebook analyzes how the COVID-19 pandemic influenced the stock price behavior 
of two major technology companies: **Apple** and **Samsung**. By combining daily stock prices 
with global daily COVID-19 case counts, this study investigates whether external global shocks 
can affect investor behavior, market volatility, and company growth patterns.

This submission corresponds to the **28 November milestone**, which includes:
- Data collection  
- Data cleaning and preprocessing  
- Exploratory data analysis (EDA)  
- Statistical hypothesis testing  


## Research Questions

- How did Apple and Samsung stock prices change between 2019 and 2024?
- Did both companies show similar growth patterns before and after the COVID-19 pandemic?
- Is there a relationship between daily COVID-19 cases and the stock prices of Apple and Samsung?
- How can the average prices and returns of Apple and Samsung be compared over time?
- Which company adapted more quickly to the market changes caused by the pandemic?

## Hypotheses

- **H1:** In the early pandemic months, Apple's stock price increased faster than Samsung's.
- **H2:** Samsung reacts more slowly to market shocks, lagging behind Apple's price growth.
- **H3:** Global COVID-19 daily case counts are positively correlated with Apple’s stock price.
- **H4:** After 2021, Samsung exhibits higher volatility compared to Apple.
- **H5:** Apple outperforms Samsung in terms of monthly average returns.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, pearsonr

plt.rcParams["figure.figsize"] = (12, 4)
sns.set(style="whitegrid")

## 1. Data Collection

In this section, Apple, Samsung, and COVID-19 datasets are loaded and merged into a single
dataframe using the `Date` column.


In [2]:
# Load CSV datasets (paths for this environment)
apple = pd.read_csv("/mnt/data/apple.csv")
samsung = pd.read_csv("/mnt/data/samsung.csv")
covid = pd.read_csv("/mnt/data/corona.csv")

# Automatically detect Samsung price column
possible_price_cols = ["Close", "Price", "Adj Close", "Adj_Close", "Closing Price"]
samsung_price_col = None

for col in possible_price_cols:
    if col in samsung.columns:
        samsung_price_col = col
        break

print("Samsung price column found:", samsung_price_col)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/apple.csv'

In [ ]:
# Fix date formats (robust)
apple["Date"] = pd.to_datetime(apple["Date"], dayfirst=True, errors="coerce")
samsung["Date"] = pd.to_datetime(samsung["Date"], dayfirst=True, errors="coerce")
covid["Date"] = pd.to_datetime(covid["Date"], dayfirst=True, errors="coerce")

# Merge datasets
df = apple.merge(samsung, on="Date", how="inner")
df = df.merge(covid, on="Date", how="left")

# Rename columns consistently
df = df.rename(columns={
    "Close": "Apple_Price",
    samsung_price_col: "Samsung_Price",
    "New cases": "Covid_Cases"
})

df.head()

## 2. Data Cleaning & Preparation

The following cleaning steps are applied:
- Convert price and COVID-19 case values into numeric format  
- Remove commas from COVID case values  
- Prepare variables for analysis  


In [ ]:
# Clean numeric columns
for col in ["Apple_Price", "Samsung_Price", "Covid_Cases"]:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "")
        .str.replace(" ", "")
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

df.describe()

## 3. Exploratory Data Analysis (EDA)

This section includes:
- Stock price trends  
- COVID-19 global case trends  
- Correlation analysis  
- Daily return distributions  


In [ ]:
# Apple vs Samsung stock prices
plt.plot(df["Date"], df["Apple_Price"], label="Apple")
plt.plot(df["Date"], df["Samsung_Price"], label="Samsung")
plt.title("Apple vs Samsung Stock Prices (2019–2024)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()

# COVID-19 daily cases
covid_part = df.dropna(subset=["Covid_Cases"])
plt.plot(covid_part["Date"], covid_part["Covid_Cases"], alpha=0.5)
plt.title("Daily COVID-19 Cases (Global, 2020)")
plt.xlabel("Date")
plt.ylabel("New Cases")
plt.show()

In [ ]:
# Correlation matrix
corr_mat = df[["Apple_Price", "Samsung_Price", "Covid_Cases"]].corr()
sns.heatmap(corr_mat, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Matrix")
plt.show()

corr_mat

In [ ]:
# Daily returns
df["Apple_Return"] = df["Apple_Price"].pct_change()
df["Samsung_Return"] = df["Samsung_Price"].pct_change()

sns.histplot(df["Apple_Return"], kde=True, label="Apple")
sns.histplot(df["Samsung_Return"], kde=True, label="Samsung")
plt.legend()
plt.title("Daily Return Distribution")
plt.show()

## 4. Hypothesis Testing

This section tests hypotheses H1–H5 using:
- Independent t-tests  
- Pearson correlations  
- Volatility (standard deviation of returns)  


In [ ]:
# H1: Early COVID period growth comparison
early_period = df[df["Date"] < "2020-06-01"]

apple_growth = early_period["Apple_Return"].dropna()
samsung_growth = early_period["Samsung_Return"].dropna()

t_stat, p_val = ttest_ind(apple_growth, samsung_growth)
print("H1 Apple vs Samsung Early Growth T-test")
print("t =", t_stat, " p =", p_val)

In [ ]:
# H2 & H5: Monthly average prices
df["Month"] = df["Date"].dt.to_period("M")
monthly = df.groupby("Month")[["Apple_Price", "Samsung_Price"]].mean().reset_index()

t_stat2, p_val2 = ttest_ind(monthly["Apple_Price"], monthly["Samsung_Price"])
print("Monthly Price Comparison T-test")
print("t =", t_stat2, " p =", p_val2)

In [ ]:
# H3: Correlation between COVID-19 cases and stock prices
apple_corr_df = df[["Covid_Cases", "Apple_Price"]].dropna()
samsung_corr_df = df[["Covid_Cases", "Samsung_Price"]].dropna()

corr_apple, p_apple = pearsonr(apple_corr_df["Covid_Cases"], apple_corr_df["Apple_Price"])
corr_samsung, p_samsung = pearsonr(samsung_corr_df["Covid_Cases"], samsung_corr_df["Samsung_Price"])

print("COVID–Apple Price Correlation:", corr_apple, "p =", p_apple)
print("COVID–Samsung Price Correlation:", corr_samsung, "p =", p_samsung)

In [ ]:
# H4: Post-2021 volatility comparison
post21 = df[df["Date"] > "2021-01-01"]

vol_apple = post21["Apple_Return"].std()
vol_samsung = post21["Samsung_Return"].std()

print("Apple Volatility (post-2021):", vol_apple)
print("Samsung Volatility (post-2021):", vol_samsung)

## 5. Summary of Findings (28 November Stage)

- Apple and Samsung stock prices show a strong positive correlation.
- Apple stock prices are strongly correlated with global COVID-19 cases, 
  while Samsung shows no meaningful correlation.
- **H1 rejected**: early-pandemic growth of Apple and Samsung is not significantly different.
- **H2 rejected** and **H5 supported**: Samsung has higher prices, but Apple demonstrates more stable returns.
- **H4 accepted**: Samsung is about 3× more volatile than Apple after 2021.

This concludes the data collection, cleaning, EDA, and hypothesis testing phase.
The next milestone (02 January) will involve machine learning models for prediction and comparison.


In [5]:
import os
import pandas as pd
import numpy as np

print("Current folder:", os.getcwd())
print("Files here:", os.listdir())

# ---- Load CSVs (same folder assumption) ----
apple = pd.read_csv("apple.csv")
samsung = pd.read_csv("samsung.csv")
covid = pd.read_csv("corona.csv")

# Detect Samsung price column
possible_price_cols = ["Close", "Price", "Adj Close", "Adj_Close", "Closing Price"]
samsung_price_col = next((c for c in possible_price_cols if c in samsung.columns), None)
print("Samsung price column:", samsung_price_col)

if samsung_price_col is None:
    raise ValueError("Samsung veri setinde fiyat kolonu bulunamadı. Kolonlar: " + str(list(samsung.columns)))

# Dates
apple["Date"] = pd.to_datetime(apple["Date"], dayfirst=True, errors="coerce")
samsung["Date"] = pd.to_datetime(samsung["Date"], dayfirst=True, errors="coerce")
covid["Date"] = pd.to_datetime(covid["Date"], dayfirst=True, errors="coerce")

# Merge
df = apple.merge(samsung, on="Date", how="inner")
df = df.merge(covid, on="Date", how="left")

# Rename
df = df.rename(columns={
    "Close": "Apple_Price",
    samsung_price_col: "Samsung_Price",
    "New cases": "Covid_Cases"
})

# Clean numeric columns
for col in ["Apple_Price", "Samsung_Price", "Covid_Cases"]:
    df[col] = (
        df[col].astype(str)
              .str.replace(",", "", regex=False)
              .str.replace(" ", "", regex=False)
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Returns
df = df.sort_values("Date").reset_index(drop=True)
df["Apple_Return"] = df["Apple_Price"].pct_change()
df["Samsung_Return"] = df["Samsung_Price"].pct_change()

print("df created ✅ shape:", df.shape)
df.head()


Current folder: /Users/aysenurucar/Downloads
Files here: ['output (2).png', 'IE 401 Inventory Simulation.py', 'ENS491_Workshop_1.pdf', 'SGK_SPASMustehaklikBelgesi.pdf', '25d2da042baa54afd9638b2cd5defd55.jpg', 'Install Spotify 2.app', '259bbb67fdbe87a07cc45801246721d3.jpg', 'd74c0356405d0d1c2a3c4d0333d5a1f6.jpg', 'FacilityLayout_2_SLP.pdf', 'EMŠO_Ayse Nur Ucar.doc', 'Random_Sentetik1.xlsx', 'Öğrenci Belgesi.pdf', 'sabanci_universitesi_logo_rgb.jpg', 'IMG_0362.PNG', 'LA_Ayse Nur Ucar.pdf', 'Transcript of Records.pdf', 'PHOTO-2024-05-14-22-43-10 10 (1).jpg', 'Sales_Management_Case_Study_Auckland_Engineering.docx', 'PHOTO-2024-05-14-22-43-10 9.jpg', 'SingleRow Layout.py', 'COURSE SATISFACTION QUESTIONNAIRE_korigiran_finalni (1).docx', 'istockphoto-496603666-612x612.jpg', 'HMC - FIN 499.pdf', 'Excel Çalışma 7.xlsx', 'PHOTO-2024-05-14-22-43-10 8.jpg', 'midterm-cheat sheet.pdf', 'Syllabus_IE304_Fall2025.pdf', 'Discrete-Event System Simulation.pdf', 'S03 - 15 Oct 2025 - Wednesday.pdf', 'f

/var/folders/gn/y5x5hzdx3xj98ddc0lk213cm0000gn/T/ipykernel_5784/1444084074.py:24: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  covid["Date"] = pd.to_datetime(covid["Date"], dayfirst=True, errors="coerce")


,Unnamed: 0,Date,Natural_Gas_Price,Natural_Gas_Vol.,Crude_oil_Price,Crude_oil_Vol.,Copper_Price,Copper_Vol.,Bitcoin_Price,Bitcoin_Vol.,...,Active,Covid_Cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries,Apple_Return,Samsung_Return
0,1242,2019-02-04,2.660,116490.0,54.56,622470.0,2.7975,490.0,"3,462.80",503920.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1241,2019-02-05,2.662,82250.0,53.66,609760.0,2.8205,90.0,"3,468.40",460950.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017286,0.003279
2,1239,2019-02-07,2.551,211790.0,52.64,749010.0,2.8320,320.0,"3,397.70",471360.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.018829,0.007625
3,1238,2019-02-08,2.583,147880.0,52.72,621000.0,2.8140,270.0,"3,661.70",699230.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003042,-0.028108
4,1236,2019-02-12,2.688,169230.0,53.10,708310.0,2.7750,50.0,"3,632.00",545820.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002817,0.121246


## 6. Machine Learning Methods (02 January)

In this section, machine learning models are applied to predict stock returns and volatility.
The goal is to compare the predictability and risk behavior of Apple and Samsung.

Two tasks are considered:
- **Task A:** Daily return prediction
- **Task B:** Volatility prediction


**A) DAILY RETURN PREDICTION - Linear Regression**

In [7]:
# ===== Feature Engineering for ML =====

df = df.sort_values("Date").reset_index(drop=True)

# Fill missing COVID values (COVID data only exists in 2020)
df["Covid_Cases_filled"] = df["Covid_Cases"].fillna(0)

# Lagged returns
df["Apple_Return_Lag1"] = df["Apple_Return"].shift(1)
df["Samsung_Return_Lag1"] = df["Samsung_Return"].shift(1)

# COVID features
df["Covid_MA7"] = df["Covid_Cases_filled"].rolling(7).mean()
df["Covid_Log"] = np.log(df["Covid_Cases_filled"] + 1)

# Pandemic dummy
df["Pandemic"] = (df["Date"] >= "2020-03-11").astype(int)

# 7-day rolling volatility targets
df["Apple_Volatility_7"] = df["Apple_Return"].rolling(7).std()
df["Samsung_Volatility_7"] = df["Samsung_Return"].rolling(7).std()

# IMPORTANT: drop NaN only for ML-relevant columns
needed_cols = [
    "Apple_Return_Lag1",
    "Samsung_Return_Lag1",
    "Covid_MA7",
    "Covid_Log",
    "Pandemic",
    "Apple_Return",
    "Samsung_Return",
    "Apple_Volatility_7",
    "Samsung_Volatility_7"
]

df_ml = df.dropna(subset=needed_cols).copy()

print("ML dataset shape:", df_ml.shape)
df_ml.head()


ML dataset shape: (370, 66)


,Unnamed: 0,Date,Natural_Gas_Price,Natural_Gas_Vol.,Crude_oil_Price,Crude_oil_Vol.,Copper_Price,Copper_Vol.,Bitcoin_Price,Bitcoin_Vol.,...,Apple_Return,Samsung_Return,Apple_Return_Lag1,Samsung_Return_Lag1,Covid_MA7,Covid_Log,Pandemic,Apple_Volatility_7,Samsung_Volatility_7,Covid_Cases_filled
7,1221,2019-03-06,2.841,119630.0,56.22,637680.0,2.9185,51450.0,"3,863.00",2520000.0,...,-0.005697,-0.033113,-0.001820,-0.027897,0.0,0.0,0,0.015775,0.061873,0.0
8,1220,2019-03-07,2.866,117000.0,56.66,628820.0,2.9105,59930.0,"3,875.10",2600000.0,...,-0.011689,0.036530,-0.005697,-0.033113,0.0,0.0,0,0.015135,0.063826,0.0
9,1218,2019-03-11,2.772,136890.0,56.79,597160.0,2.9010,43180.0,"3,870.30",3190000.0,...,0.037338,0.127753,-0.011689,0.036530,0.0,0.0,0,0.018760,0.080222,0.0
10,1217,2019-03-12,2.784,90020.0,56.87,652800.0,2.9285,68210.0,"3,886.00",2900000.0,...,0.011178,-0.025391,0.037338,0.127753,0.0,0.0,0,0.018292,0.079972,0.0
11,1201,2019-04-03,2.677,96800.0,62.46,799670.0,2.9525,350.0,"4,968.70",13830000.0,...,0.079814,-0.101202,0.011178,-0.025391,0.0,0.0,0,0.032088,0.076058,0.0


**Feature Selection**

The following variables are used as inputs for the machine learning model:

In [11]:
features = [
    "Apple_Return_Lag1",
    "Samsung_Return_Lag1",
    "Covid_MA7",
    "Covid_Log",
    "Pandemic"
]


**Train–Test Split**

To evaluate out-of-sample performance, the dataset is split chronologically:

Training set: data before 2022

Test set: data from 2022 onward

In [12]:
split_date = "2022-01-01"

train = df_ml[df_ml["Date"] < split_date]
test  = df_ml[df_ml["Date"] >= split_date]

X_train = train[features]
X_test  = test[features]


**Linear Regression Model**

A linear regression model is trained to predict Apple’s daily returns.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

y_train = train["Apple_Return"]
y_test  = test["Apple_Return"]

lr = LinearRegression()
lr.fit(X_train, y_train)

pred = lr.predict(X_test)

print("Apple Return Prediction (Linear Regression)")
print("MAE:", mean_absolute_error(y_test, pred))
print("R2 :", r2_score(y_test, pred))


**Section A Results Interpretation**

The typical size of daily forecast mistakes is indicated by the model's Mean Absolute Error (MAE), which is roughly 2-3%.
Daily stock returns are very noisy and challenging to forecast using linear models, according to the negative R2 value. This outcome is in line with financial literature, which acknowledges the difficulty of predicting short-term stock returns.

Overall, rather than focusing on predicted accuracy, this section illustrates the use of machine learning approaches, feature engineering, and appropriate out-of-sample evaluation.

**B) APPLE RETURN PREDICTION (Random Forest)**

In [13]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=5,
    random_state=42
)

rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)

print("Apple Return Prediction (Random Forest)")
print("MAE:", mean_absolute_error(y_test, rf_pred))
print("R2 :", r2_score(y_test, rf_pred))


Apple Return Prediction (Random Forest)
MAE: 0.0259601157224783
R2 : -0.2558433359122807


**Model Comparison**

To capture possible non-linear correlations, a Random Forest model was used in addition to linear regression.
The noisy character of daily stock returns is shown by the Random Forest model's low predictive accuracy, despite a minor improvement in flexibility.
As required by the milestone, this comparison illustrates the application of many machine learning techniques.